In [ ]:
from IPython.display import display, Image, clear_output
import cv2

width, height = 640, 480
pipeline = (
    f"nvarguscamerasrc ! "
    f"video/x-raw(memory:NVMM), width={width}, height={height}, format=NV12, framerate=30/1 ! "
    f"nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)
camera = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)

if not camera.isOpened():
    raise RuntimeError("Không mở được camera CSI")

while True:
    ret, frame = camera.read()
    if not ret:
        continue
    _, jpeg = cv2.imencode('.jpg', frame)
    display(Image(data=jpeg.tobytes()))
    clear_output(wait=True)
